In [2]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
DATASET_DIR = "images"

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels='inferred',
    label_mode='int',
    class_names=['spiral', 'elliptical'],
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels='inferred',
    label_mode='int',
    class_names=['spiral', 'elliptical'],
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = ['spiral', 'elliptical']
print("Classes:", class_names)

Found 100 files belonging to 2 classes.
Using 80 files for training.
Found 100 files belonging to 2 classes.
Using 20 files for validation.
Classes: ['spiral', 'elliptical']


In [10]:
# Optimize dataset
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:

# Build CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

C:\Users\tucke\anaconda3\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
# Train model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 207ms/step - accuracy: 0.5335 - loss: 0.7200 - val_accuracy: 0.4000 - val_loss: 0.7112
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.6148 - loss: 0.6335 - val_accuracy: 0.6000 - val_loss: 0.6483
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.7762 - loss: 0.5306 - val_accuracy: 0.7500 - val_loss: 0.5068
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.8849 - loss: 0.2874 - val_accuracy: 0.8500 - val_loss: 0.5963
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.9403 - loss: 0.1533 - val_accuracy: 0.7500 - val_loss: 0.7799
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.9469 - loss: 0.1098 - val_accuracy: 0.7000 - val_loss: 0.9474
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.9856 - loss: 0.0463 - val_accuracy: 0.6000 - val_loss: 1.3386
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.9845 - loss: 0.0494 - val_accuracy: 0.6500 - val_loss:

In [18]:
final_train_loss, final_train_acc = model.evaluate(train_ds)
final_val_loss, final_val_acc = model.evaluate(val_ds)

print(f"Final Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc * 100:.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0178
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5667 - loss: 1.9507
Final Training Accuracy: 100.00%
Final Validation Accuracy: 60.00%


In [26]:
def predict_image(image_path, threshold=0.6):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = model.predict(img_array)[0]

    # Get top prediction and confidence
    top_index = np.argmax(predictions)
    top_confidence = predictions[top_index]
    predicted_class = class_names[top_index]

    # Fallback to irregular if not confident
    if top_confidence < threshold:
        predicted_class = 'irregular'

    print(f"Prediction: {predicted_class} (Confidence: {top_confidence:.2f})")
    return predicted_class

In [33]:
predict_image("988001.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Prediction: elliptical (Confidence: 1.00)


'elliptical'

In [35]:
predict_image("987261.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: spiral (Confidence: 0.85)


'spiral'

In [41]:
predict_image("images (2).jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Prediction: spiral (Confidence: 1.00)


'spiral'

In [53]:
predict_image("image0.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Prediction: elliptical (Confidence: 0.90)


'elliptical'

In [55]:
predict_image("M104_ngc4594_sombrero_galaxy_hi-res.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Prediction: spiral (Confidence: 1.00)


'spiral'

In [57]:
predict_image("Kirby_Nintendo.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Prediction: spiral (Confidence: 1.00)


'spiral'